In [101]:
import os
import pandas as pd
import numpy as np
import pickle

# Setup

In [102]:
data_dir = os.path.join(os.curdir, "data")

raw_dataset_url = "https://shai-nlp-course.netlify.app/clean-tweets.tsv"

In [103]:
raw = pd.read_csv(filepath_or_buffer=raw_dataset_url, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,أؤمن بأن الانسان ينطفئ جماله ابتعاد يحب بريق ا...,اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياة فنحن نعيش أرض أعدت للبلاء و...,خلو ضغط حية فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كآنت نقيه,نصم تسر حيت يرم لنس تعد كآن نقه


In [104]:
tweets = raw["clean_stemmed"]

# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [105]:
sample = tweets.iloc[0]
sample

'اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذبل طفئ تحل ربع خرف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [106]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str]):
        self.vocab = self._create_vocab(corpus=corpus)
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
    
    def _tokenize_document(self, document: str) -> list[int]:
        ...
    
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        return [self._tokenize_document(document) for document in documents]


## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [107]:
sample_tokens = sample.split(" ")
sample_tokens

['اؤم',
 'بأن',
 'انس',
 'طفئ',
 'جمل',
 'بعد',
 'يحب',
 'برق',
 'عين',
 'خفي',
 'صبح',
 'ذبل',
 'طفئ',
 'تحل',
 'ربع',
 'خرف']

In [108]:
# Create a list of all unique tokens in the corpus
all_word_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(all_word_tokens)
print(len(word_level_tokens), len(all_word_tokens))

7754 61656


In [109]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'ماضيمستحيل': 0,
 'خرىهل': 1,
 'سيف': 2,
 'مانحتاج': 3,
 'صال': 4,
 '😂😂😜': 5,
 'أدمنو': 6,
 'بأد': 7,
 'لعو': 8,
 'عث…': 9,
 'طيح': 10,
 'فهس': 11,
 'شيخ': 12,
 'اسبب': 13,
 'بكف': 14,
 'ماجابر': 15,
 'سهم': 16,
 'تشك': 17,
 'قوش': 18,
 'بطن': 19,
 'هـى': 20,
 'كدر': 21,
 'ﻭأﻧﺖ': 22,
 'تمز': 23,
 'ثوبي…': 24,
 'ضهر': 25,
 'طار': 26,
 'لتر': 27,
 'محمدقدم': 28,
 'حرف❤️': 29,
 'وهج': 30,
 'وصو': 31,
 'لأح': 32,
 'هئل': 33,
 'شيز': 34,
 'انك': 35,
 'لقت': 36,
 'خلت': 37,
 'غفر': 38,
 'سه…': 39,
 'تحك': 40,
 'جمعو': 41,
 'جلد': 42,
 'غدق': 43,
 'صيء': 44,
 'عاها…': 45,
 'ثأر': 46,
 'فخم': 47,
 'ندم': 48,
 'و…': 49,
 'اقصر': 50,
 'عودي…': 51,
 'زدي': 52,
 'اةهالازمة': 53,
 'سحه': 54,
 'نيز': 55,
 'دره': 56,
 'ديب': 57,
 '«خط': 58,
 'احط': 59,
 'ديوان…': 60,
 'كرهو': 61,
 'يفز': 62,
 'عكن': 63,
 'خون': 64,
 'ر😊😊': 65,
 'يعف': 66,
 'بغب': 67,
 'شرى': 68,
 'لير': 69,
 '👩لش': 70,
 'لمب': 71,
 'اخة': 72,
 'محسني…': 73,
 'جينوم': 74,
 'دحه': 75,
 'علا': 76,
 'كرم': 77,
 'جرد': 78,
 'تئه': 79,
 '

In [110]:
class WordLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str]):
        super().__init__(corpus=corpus)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        all_word_tokens = set([token for sample in corpus for token in sample.split(" ")])
        vocab = {token: index for index, token in enumerate(word_level_tokens, start=1)} 
        vocab["[PAD]"] = 0
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document.split(" ")]

    

In [111]:
word_level_tokenizer = WordLevelTokenizer(corpus=tweets)

## Character Level Tokenization

In [112]:
sample_tokens = list(sample)
sample_tokens

['ا',
 'ؤ',
 'م',
 ' ',
 'ب',
 'أ',
 'ن',
 ' ',
 'ا',
 'ن',
 'س',
 ' ',
 'ط',
 'ف',
 'ئ',
 ' ',
 'ج',
 'م',
 'ل',
 ' ',
 'ب',
 'ع',
 'د',
 ' ',
 'ي',
 'ح',
 'ب',
 ' ',
 'ب',
 'ر',
 'ق',
 ' ',
 'ع',
 'ي',
 'ن',
 ' ',
 'خ',
 'ف',
 'ي',
 ' ',
 'ص',
 'ب',
 'ح',
 ' ',
 'ذ',
 'ب',
 'ل',
 ' ',
 'ط',
 'ف',
 'ئ',
 ' ',
 'ت',
 'ح',
 'ل',
 ' ',
 'ر',
 'ب',
 'ع',
 ' ',
 'خ',
 'ر',
 'ف']

In [113]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

346


In [114]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'ظ': 0,
 'ئ': 1,
 'ﻋ': 2,
 '🍀': 3,
 'ﻙ': 4,
 'و': 5,
 'ﻭ': 6,
 '🇲': 7,
 '👮': 8,
 '😷': 9,
 'ﻢ': 10,
 'ﻱ': 11,
 '😤': 12,
 'ر': 13,
 '💡': 14,
 'ﺤ': 15,
 'ﻤ': 16,
 '🇧': 17,
 'ؤ': 18,
 ' ': 19,
 'ٓ': 20,
 '🏽': 21,
 '⚘': 22,
 '😐': 23,
 '⸀': 24,
 '﴾': 25,
 '♂': 26,
 'ﻫ': 27,
 'แ': 28,
 '🐳': 29,
 'ﺒ': 30,
 '🚶': 31,
 '”': 32,
 'ث': 33,
 'ء': 34,
 '💃': 35,
 'أ': 36,
 'ﻣ': 37,
 '🕖': 38,
 '🌝': 39,
 'ﺗ': 40,
 '👏': 41,
 'ڪ': 42,
 '⌛': 43,
 'ش': 44,
 'ﺘ': 45,
 '\u200c': 46,
 '﷽': 47,
 '🤯': 48,
 '↴': 49,
 'ﻮ': 50,
 '🇪': 51,
 'ا': 52,
 '🤬': 53,
 '😡': 54,
 '💔': 55,
 '💵': 56,
 '😑': 57,
 'ﺀ': 58,
 '🙈': 59,
 '💚': 60,
 'ح': 61,
 'ﻥ': 62,
 '👌': 63,
 'ب': 64,
 'ﻜ': 65,
 '💪': 66,
 '😕': 67,
 'ۈ': 68,
 '»': 69,
 '🔺': 70,
 '💎': 71,
 '😠': 72,
 'چ': 73,
 '💳': 74,
 'ﻔ': 75,
 '٪': 76,
 '😬': 77,
 '🇱': 78,
 '🙄': 79,
 '😩': 80,
 'ﻳ': 81,
 '💖': 82,
 '🙂': 83,
 '￼': 84,
 'ﻐ': 85,
 '🎂': 86,
 '🤦': 87,
 '👆': 88,
 '😻': 89,
 '😊': 90,
 '🖤': 91,
 '😆': 92,
 '☕': 93,
 '🤷': 94,
 'ﻧ': 95,
 'ﺱ': 96,
 '😶': 97,
 '״': 98,
 '’': 99,
 'ﻲ':

In [115]:
# TODO: Implement character level tokenizer
# 1. __init__()
# 2. _create_vocab
# 3. _tokenize_document 

class CharacterLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str]):
        super().__init__(corpus=corpus)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        all_word_tokens = set([token for sample in corpus for token in sample])
        vocab = {token: index for index, token in enumerate(word_level_tokens, start=1)} 
        vocab["[PAD]"] = 0
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document]

    

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

1. List of documents (corpus)
2. Each document is represented by a sequence of tokens

> Not all documents have the same length

In [116]:
tokenized_tweets = word_level_tokenizer.tokenize(documents=tweets)

In [117]:
max([len(t) for t in tokenized_tweets])

25

In [118]:
characterLevelTokenizer = CharacterLevelTokenizer(corpus=tweets)
tokenized_tweets_chars = characterLevelTokenizer.tokenize(documents=tweets)


In [119]:
max([len(t) for t in tokenized_tweets_chars])

106

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization

In [120]:
word_level_tokenizer_path = os.path.join(data_dir, "word_tokenizer.pkl")

with open(word_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=word_level_tokenizer, file=f)


In [121]:
char_level_tokenizer_path = os.path.join(data_dir, "char_tokenizer.pkl")

with open(char_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=characterLevelTokenizer, file=f)